## Ucitavanje baze

In [1]:
from funkcije import *

In [2]:
baza_location = biraj_bazu()

FileChooser(path='/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav', filen…

In [3]:
baza = ucitaj_bazu(baza_location)

greska sa citanjem fajla:/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav/newbaza


/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/funkcije.py:132: UserWarning: PySoundFile failed. Trying audioread instead.
  data, samplerate = librosa.load(fc.selected + filename)
/home/zarko/.local/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [4]:
#liste ficera 
hurstovi = [] #+
duzine = [] #+
odnostig = [] #+
najduzaprica = [] #+

zcrovi = [] #lokalni
pitchovi = []

zcr_prosek = [] #globalni #+
zcr_min = []
zcr_max = []
zcr_med = []
zcr_stdev = []

pitch_prosek = [] #globalni 
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

energije_ukupne = []
energije_short_term = []
energije_short_term_silence = []

eng_prosek = [] #globalni 
eng_min = []
eng_max = []
eng_med = []
eng_stdev = []

engbeztis_prosek = [] #globalni 
engbeztis_min = []
engbeztis_max = []
engbeztis_med = []
engbeztis_stdev = []

frame_lenght = 480 
hop_lenght = frame_lenght//2

mfccovi = []
mfccovi_array = []


frame_length_tisina = 100
hop_length_tisina = frame_length_tisina//2
granica = 0.01

---
## hurst 

In [7]:

for sample in baza:
        h, c, val = compute_Hc(sample.data)
        hurstovi.append(h)

### fing imenovanje

        mfcc_f_one_of_twenty.append(np.mean(matrica[j]))
        mfcc_f_one_of_twenty.append(np.min(matrica[j]))
        mfcc_f_one_of_twenty.append(np.max(matrica[j]))
        mfcc_f_one_of_twenty.append(np.median(matrica[j]))
        mfcc_f_one_of_twenty.append(np.var(matrica[j]))

In [7]:
for i in range (1, 21):
    print( "," + "mfcc_mean_" + str(i), end = ' ')
    print( "," + "mfcc_min_" + str(i), end = ' ')
    print( "," + "mfcc_max_" + str(i), end = ' ')
    print( "," + "mfcc_med_" + str(i), end = ' ')
    print( "," + "mfcc_var_" + str(i), end = ' ')

mfcc_mean_1 ,mfcc_min_1 ,mfcc_max_1 ,mfcc_med_1 ,mfcc_var_1 mfcc_mean_2 ,mfcc_min_2 ,mfcc_max_2 ,mfcc_med_2 ,mfcc_var_2 mfcc_mean_3 ,mfcc_min_3 ,mfcc_max_3 ,mfcc_med_3 ,mfcc_var_3 mfcc_mean_4 ,mfcc_min_4 ,mfcc_max_4 ,mfcc_med_4 ,mfcc_var_4 mfcc_mean_5 ,mfcc_min_5 ,mfcc_max_5 ,mfcc_med_5 ,mfcc_var_5 mfcc_mean_6 ,mfcc_min_6 ,mfcc_max_6 ,mfcc_med_6 ,mfcc_var_6 mfcc_mean_7 ,mfcc_min_7 ,mfcc_max_7 ,mfcc_med_7 ,mfcc_var_7 mfcc_mean_8 ,mfcc_min_8 ,mfcc_max_8 ,mfcc_med_8 ,mfcc_var_8 mfcc_mean_9 ,mfcc_min_9 ,mfcc_max_9 ,mfcc_med_9 ,mfcc_var_9 mfcc_mean_10 ,mfcc_min_10 ,mfcc_max_10 ,mfcc_med_10 ,mfcc_var_10 mfcc_mean_11 ,mfcc_min_11 ,mfcc_max_11 ,mfcc_med_11 ,mfcc_var_11 mfcc_mean_12 ,mfcc_min_12 ,mfcc_max_12 ,mfcc_med_12 ,mfcc_var_12 mfcc_mean_13 ,mfcc_min_13 ,mfcc_max_13 ,mfcc_med_13 ,mfcc_var_13 mfcc_mean_14 ,mfcc_min_14 ,mfcc_max_14 ,mfcc_med_14 ,mfcc_var_14 mfcc_mean_15 ,mfcc_min_15 ,mfcc_max_15 ,mfcc_med_15 ,mfcc_var_15 mfcc_mean_16 ,mfcc_min_16 ,mfcc_max_16 ,mfcc_med_16 ,mfcc_var_16 mfcc_

### duzine

In [8]:


for sample in baza:

     duzina = len(sample.data) * 1/sample.samplerate
        
     duzine.append(duzina) 

In [9]:
### odnos govor prema tisina, max govor 

duzinatisine = []

#  frejmovano


for sample in baza:
      
     duzina = len(sample.data) * 1/sample.samplerate
     duzinatisinepriv = 0
     ptisina = 0
     trenprica = 0
     maxprica = 0
     for i in range (0, len(sample.data), hop_length_tisina):
          engparceta = sum(abs(sample.data[i:i+frame_length_tisina]**2))
          if engparceta < granica:
               trenprica = 0
               if ptisina == 0:
                    duzinatisinepriv = duzinatisinepriv + frame_length_tisina * 1/sample.samplerate
               else :
                    duzinatisinepriv = duzinatisinepriv + hop_length_tisina * 1/sample.samplerate
               ptisina = 1
          else: 
               ptisina = 0
               if trenprica == 0:
                    trenprica = frame_length_tisina * 1/sample.samplerate
               else :
                    trenprica = trenprica + hop_length_tisina * 1/sample.samplerate
               if trenprica > maxprica: maxprica = trenprica
     duzinatisine.append(duzinatisinepriv)
     najduzaprica.append(maxprica)

for i in range (0, len(duzine)):
     if duzinatisine[i] > 0:
          odnostig.append((duzine[i]-duzinatisine[i])/duzinatisine[i])
     else: odnostig.append(1)      

## MFCC
---

In [63]:
# data # citav audio fajl u nizu
# parcici[0] # prvi segment tog audio fajla(mozda nije prvi zbog tisine zapravo) --- NE
# prvi_seg # prvi segment koji nije tisina(trenutno su ostali pobrisani ovde)

mfccovi_array = []
mfcc_f_one_of_twenty = []

for sample in baza:
            
    matrica = librosa.feature.mfcc (y = sample.data, sr = sample.samplerate)  ##20 x nesto 
    matrica = sklearn.preprocessing.scale(matrica, axis=1)
    # prvi set: mean max itd od matrica[0]
    #mfcc_ficeri_za_jedan = np.array ([20,6])
    mfcc_f_one_of_twenty = []

        


    


IndentationError: expected an indented block (1584024443.py, line 22)

In [11]:
mfccovi_sredjeni = []

for i in range (0, len(mfccovi)):
    priv = sklearn.preprocessing.scale(mfccovi[i], axis=1)
    mfccovi_sredjeni.append(priv)

535


In [27]:
i = 0
mfcc_f_one_of_twenty = []
mfcc_f_for_one_file = []
mfcc_f_all = []

for sample in baza:
    matrica = librosa.feature.mfcc(y = sample.data, sr = sample.samplerate) #20xnesto
    matrica = sklearn.preprocessing.scale(matrica, axis=1)
    for j in range (0, 20):
        mfcc_f_one_of_twenty.append(np.mean(matrica[j]))
        mfcc_f_one_of_twenty.append(np.min(matrica[j]))
        mfcc_f_one_of_twenty.append(np.max(matrica[j]))
        mfcc_f_one_of_twenty.append(np.median(matrica[j]))
        mfcc_f_one_of_twenty.append(np.var(matrica[j]))
     #napravi se lista od 6 ficera 
        mfcc_f_for_one_file.append(mfcc_f_one_of_twenty) #napravi se lista od 20x6 ficera 
    mfcc_f_all.append(mfcc_f_for_one_file) #lista 535x20x6 ficera za sve fajlove 

/home/zarko/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/zarko/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/zarko/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/zarko/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard dev

In [29]:

i = 0
f = open ("baza_norm_with_mfcc.csv", "w")
for sample in baza:
    
    f.write(str(energije_ukupne_norm[0][i]) + ",")
    tren = mfcc_f_all[i] #20x6
    for j in range (0, 20):
        priv = tren[j]
        for l in range (0, 5):
            f.write(str(priv[l]) + ",")
    f.write("\n")
    i = i+1

f.close()

In [28]:
priv = tren[0]
for l in range (0, 5):
    print(str(priv[l]) + ",")

5.815087e-09,
-2.4696302,
1.7650337,
0.20490937,
1.0,


## ZCR
---

In [13]:
## lokalni 


for sample in baza:
        #pitch = librosa.feature.zero_crossing_rate(data, frame_length = frame_length, hop_length = hop_length)
        zcr = librosa.feature.zero_crossing_rate(sample.data, frame_length = frame_lenght, hop_length = hop_lenght)
        #print(pitch)
        #print(zcr[0])         
        #plt.plot(zcr)      #vraca matricu iz nekog razloga, ali je sav sadrzaj u zcr[0] tkd cu samo to da uzimam
        zcrovi.append(zcr[0])

In [14]:
## globalni 

for zcr in zcrovi:
    zcr_prosek.append(np.mean(zcr))
    zcr_min.append(np.min(zcr))
    zcr_max.append(np.max(zcr))
    zcr_med.append(np.median(zcr))
    zcr_stdev.append(np.std(zcr))
    #print (prosek, min, max, med, stdev)
    #break

## F0 --- pitch
---

In [15]:
#lokalni

for sample in baza:

    f0, voiced_flag, voiced_probs = librosa.pyin(sample.data,

                                                fmin=librosa.note_to_hz('C2'),

                                                fmax=librosa.note_to_hz('C7'), sr=sample.samplerate) #, frame_length=frame_len, hop_length=hop_len)
    pitchovi.append(f0)

In [16]:
## globalni 

pitch_prosek = []
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

for pitch_nn in pitchovi:
    pitch = pitch_nn[pitch_nn > -1]
    pitch_prosek.append(np.mean(pitch))
    pitch_min.append(np.min(pitch))
    pitch_max.append(np.max(pitch))
    pitch_med.append(np.median(pitch))
    pitch_stdev.append(np.std(pitch))
    #print (prosek, min, max, med, stdev)
    #break

## energija
---

In [7]:
# energija


# short term energija, frejmovano
frame_length = frame_length_tisina # videcemo ali ovo ce ukloniti vise tisine tkd trebalo bi da utice 
hop_length = frame_length//2


for sample in baza:

    energy = sum(abs(sample.data**2))  ## ukupna energija fajla
    energije_ukupne.append(energy)

    energy_arr = np.array([
        sum(abs( sample.data[i:i+frame_length]**2))
        for i in range(0, len(sample.data), hop_length)])

    energije_short_term_silence.append(energy_arr)
    
for energija_kratko in energije_short_term_silence:
    energije_short_term.append(energija_kratko[energija_kratko > granica])

In [30]:


for eng in energije_short_term:
    eng_prosek.append(np.mean(eng))
    eng_min.append(np.min(eng))
    eng_max.append(np.max(eng))
    eng_med.append(np.median(eng))
    eng_stdev.append(np.std(eng))

for engbeztis in energije_short_term_silence:
    engbeztis_prosek.append(np.mean(engbeztis))
    engbeztis_min.append(np.min(engbeztis))
    engbeztis_max.append(np.max(engbeztis))
    engbeztis_med.append(np.median(engbeztis))
    engbeztis_stdev.append(np.std(engbeztis)) 



---
## normalizacija

In [22]:

hurstovi_norm = preprocessing.normalize([np.asarray(hurstovi)])
duzine_norm = preprocessing.normalize([np.asarray(duzine)])
odnostig_norm = preprocessing.normalize([np.asarray(odnostig)])
najduzaprica_norm = preprocessing.normalize([np.asarray(najduzaprica)])

zcr_prosek_norm = preprocessing.normalize([np.asarray(zcr_prosek)])
zcr_min_norm = preprocessing.normalize([np.asarray(zcr_min)])
zcr_max_norm = preprocessing.normalize([np.asarray(zcr_max)])
zcr_med_norm = preprocessing.normalize([np.asarray(zcr_med)])
zcr_stdev_norm = preprocessing.normalize([np.asarray(zcr_stdev)])

pitch_prosek_norm = preprocessing.normalize([np.asarray(pitch_prosek)])
pitch_min_norm = preprocessing.normalize([np.asarray(pitch_min)])
pitch_max_norm = preprocessing.normalize([np.asarray(pitch_max)])
pitch_med_norm = preprocessing.normalize([np.asarray(pitch_med)])
pitch_stdev_norm = preprocessing.normalize([np.asarray(pitch_stdev)])

energije_ukupne_norm = preprocessing.normalize([np.asarray(energije_ukupne)])

eng_prosek_norm = preprocessing.normalize([np.asarray(eng_prosek)])
eng_min_norm = preprocessing.normalize([np.asarray(eng_min)])
eng_max_norm = preprocessing.normalize([np.asarray(eng_max)])
eng_med_norm = preprocessing.normalize([np.asarray(eng_med)])
eng_stdev_norm = preprocessing.normalize([np.asarray(eng_stdev)])

engbeztis_prosek_norm = preprocessing.normalize([np.asarray(engbeztis_prosek)])
engbeztis_min_norm = preprocessing.normalize([np.asarray(engbeztis_min)])
engbeztis_max_norm = preprocessing.normalize([np.asarray(engbeztis_max)])
engbeztis_med_norm = preprocessing.normalize([np.asarray(engbeztis_med)])
engbeztis_stdev_norm = preprocessing.normalize([np.asarray(engbeztis_stdev)])


In [11]:
energije_ukupne_norm = preprocessing.normalize([np.asarray(energije_ukupne)])

---
## ucitavanje ficera u fajlove 

### ficeri_glb

In [ ]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "w")
    f.close()

hurstovi;
duzine;
odnostig;
najduzaprica;
zcr_prosek;
zcr_min;
zcr_max;
zcr_med;
zcr_stdev;
pitch_prosek;
pitch_min;
pitch_max;
pitch_med;
pitch_stdev;

filename, govornik, recenica, emocija, emo, pol, godiste

In [ ]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "a")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi_norm[i]) + ",")

    f.write(str(duzine_norm[i]) + ",")
    f.write(str(odnostig_norm[i]) + ",")
    f.write(str(najduzaprica_norm[i]) + ",")

    f.write(str(zcr_prosek_norm[i]) + ",")
    f.write(str(zcr_min_norm[i]) + ",")
    f.write(str(zcr_max_norm[i]) + ",")
    f.write(str(zcr_med_norm[i]) + ",")
    f.write(str(zcr_stdev_norm[i]) + ",")

    f.write(str(pitch_prosek_norm[i]) + ",")
    f.write(str(pitch_min_norm[i]) + ",")
    f.write(str(pitch_max_norm[i]) + ",")
    f.write(str(pitch_med_norm[i]) + ",")
    f.write(str(pitch_stdev_norm[i]) + ",")
    
    f.close()
    i = i+1

### ficeri_glb_norm

In [20]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb_norm/" + name + ".txt", "w")
    f.close()

In [28]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb_norm/" + name + ".txt", "w")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi_norm[0][i]) + ",")

    f.write(str(duzine_norm[0][i]) + ",")
    f.write(str(odnostig_norm[0][i]) + ",")
    f.write(str(najduzaprica_norm[0][i]) + ",")

    f.write(str(zcr_prosek_norm[0][i]) + ",")
    f.write(str(zcr_min_norm[0][i]) + ",")
    f.write(str(zcr_max_norm[0][i]) + ",")
    f.write(str(zcr_med_norm[0][i]) + ",")
    f.write(str(zcr_stdev_norm[0][i]) + ",")

    f.write(str(pitch_prosek_norm[0][i]) + ",")
    f.write(str(pitch_min_norm[0][i]) + ",")
    f.write(str(pitch_max_norm[0][i]) + ",")
    f.write(str(pitch_med_norm[0][i]) + ",")
    f.write(str(pitch_stdev_norm[0][i]) + ",")

    f.write(str(eng_prosek_norm[0][i]) + ",")
    f.write(str(eng_min_norm[0][i]) + ",")
    f.write(str(eng_max_norm[0][i]) + ",")
    f.write(str(eng_med_norm[0][i]) + ",")
    f.write(str(eng_stdev_norm[0][i]) + ",")

    f.write(str(engbeztis_prosek_norm[0][i]) + ",")
    f.write(str(engbeztis_min_norm[0][i]) + ",")
    f.write(str(engbeztis_max_norm[0][i]) + ",")
    f.write(str(engbeztis_med_norm[0][i]) + ",")
    f.write(str(engbeztis_stdev_norm[0][i]) + ",")
    
    f.close()
    i = i+1

In [ ]:

i = 0
f = open ("baza_norm.with_mfcccsv", "w")
for sample in baza:
    
    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")

    f.write(str(hurstovi_norm[0][i]) + ",")

    f.write(str(duzine_norm[0][i]) + ",")
    f.write(str(odnostig_norm[0][i]) + ",")
    f.write(str(najduzaprica_norm[0][i]) + ",")

    f.write(str(zcr_prosek_norm[0][i]) + ",")
    f.write(str(zcr_min_norm[0][i]) + ",")
    f.write(str(zcr_max_norm[0][i]) + ",")
    f.write(str(zcr_med_norm[0][i]) + ",")
    f.write(str(zcr_stdev_norm[0][i]) + ",")

    f.write(str(pitch_prosek_norm[0][i]) + ",")
    f.write(str(pitch_min_norm[0][i]) + ",")
    f.write(str(pitch_max_norm[0][i]) + ",")
    f.write(str(pitch_med_norm[0][i]) + ",")
    f.write(str(pitch_stdev_norm[0][i]) + ",")

    f.write(str(eng_prosek_norm[0][i]) + ",")
    f.write(str(eng_min_norm[0][i]) + ",")
    f.write(str(eng_max_norm[0][i]) + ",")
    f.write(str(eng_med_norm[0][i]) + ",")
    f.write(str(eng_stdev_norm[0][i]) + ",")

    f.write(str(engbeztis_prosek_norm[0][i]) + ",")
    f.write(str(engbeztis_min_norm[0][i]) + ",")
    f.write(str(engbeztis_max_norm[0][i]) + ",")
    f.write(str(engbeztis_med_norm[0][i]) + ",")
    f.write(str(engbeztis_stdev_norm[0][i]))


    velicina = str(mfccovi[i].shape)[1:3] + "," + str(mfccovi[i].shape)[5:7]
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
        f.write("\n")
    
    i = i+1

f.close()_

In [47]:

i = 0
f = open ("baza_norm_with_mfcc.csv", "w")
for sample in baza:
    
    f.write(str(energije_ukupne_norm[0][i]))

    velicina = "," + str(mfccovi[i].shape[0]) + "," + str(mfccovi[i].shape[1])
    # print(velicina)
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
    f.write("\n")
    
    i = i+1

f.close()

### ficeri_mfcc

In [ ]:

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    f.close()

In [1]:
i = 0

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    velicina = str(mfccovi[i].shape[0]) + "," + str(mfccovi[i].shape[1])
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
    f.close()
    i=i+1

NameError: name 'baza' is not defined

In [ ]:
print(velicina)
#print(velicina)
#print(velicina[:1] + "," + velicina[1:] + ",")

In [ ]:
print()

## Ucitavanje svega nazad u nizove
---

In [38]:
df = pd.read_csv("baza_full.csv")

In [58]:
print(zcr_prosek)

[0.0396268326626213, 0.0275233956105626, 0.0508636679127195, 0.0427650608894762, 0.0276080740439196, 0.0438978623762858, 0.0550140292559772, 0.0519262548347004, 0.0564783002177407, 0.0463540807486027, 0.0328880104442484, 0.032803623581269, 0.0300580447001686, 0.0563715307084447, 0.0524044489474793, 0.0457953339062536, 0.0388059943962144, 0.0359895751141673, 0.0560505736141403, 0.0563561854958287, 0.0492094937347097, 0.0504551727730828, 0.0369669740573828, 0.0340942723941214, 0.0615249646119888, 0.0455933604676498, 0.0406040493531289, 0.038954688435423, 0.0346048224331777, 0.0524262060595109, 0.053180310686207, 0.0467045745190615, 0.033227561540157, 0.0362678444323599, 0.0261838502072906, 0.0434958376891176, 0.0334916759518067, 0.0252305105027086, 0.0467798413442322, 0.0221002855134345, 0.0305152732898607, 0.0242696437951389, 0.043130238239159, 0.0477350309289072, 0.0385207491151209, 0.0314026393733825, 0.0332496890161868, 0.0452573714880014, 0.051470193667603, 0.0401865428277725, 0.031

In [57]:
niz_svega = df.to_numpy()
for i in range(len(niz_svega)):
    hurstovi.append(niz_svega[i][7])
    duzine.append(niz_svega[i][8])
    
    odnostig.append(niz_svega[i][9])
    najduzaprica.append(niz_svega[i][10])
    zcr_prosek.append(niz_svega[i][11])
    zcr_min.append(niz_svega[i][12])
    zcr_max.append(niz_svega[i][13])
    zcr_med.append(niz_svega[i][14])
    zcr_stdev.append(niz_svega[i][15])
    
    pitch_prosek.append(niz_svega[i][16])
    pitch_min.append(niz_svega[i][17])
    pitch_max.append(niz_svega[i][18])
    pitch_med.append(niz_svega[i][19])
    pitch_stdev.append(niz_svega[i][20])

    eng_prosek.append(niz_svega[i][21])
    eng_min.append(niz_svega[i][22])
    eng_max.append(niz_svega[i][23])
    eng_med.append(niz_svega[i][24])
    eng_stdev.append(niz_svega[i][25])

    engbeztis_prosek.append(niz_svega[i][26])
    engbeztis_min.append(niz_svega[i][27])
    engbeztis_max.append(niz_svega[i][28])
    engbeztis_med.append(niz_svega[i][29])
    engbeztis_stdev.append(niz_svega[i][30])

    energije_ukupne.append(niz_svega[i][31])

    mfcc_x.append(niz_svega[i][32])
    mfcc_y.append(niz_svega[i][33])
    


    

In [55]:
# majko mila
#liste ficera 
hurstovi = [] #+
duzine = [] #+
odnostig = [] #+
najduzaprica = [] #+

# mfccovi = []

# zcrovi = [] #lokalni
# pitchovi = []

zcr_prosek = [] #globalni #+
zcr_min = []
zcr_max = []
zcr_med = []
zcr_stdev = []

pitch_prosek = [] #globalni 
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

energije_ukupne = []
energije_short_term = []
energije_short_term_silence = []

eng_prosek = [] #globalni 
eng_min = []
eng_max = []
eng_med = []
eng_stdev = []

engbeztis_prosek = [] #globalni 
engbeztis_min = []
engbeztis_max = []
engbeztis_med = []
engbeztis_stdev = []

mfcc_x = []
mfcc_y = []


In [56]:
df = pd.read_csv("baza_full.csv")

In [49]:
mfccovi[0].shape[0]

20

In [50]:
df

,filename,kod govornika,kod recenice,kod emocije,naziv emocije,pol,godiste,hurstovi,duzine,odnos tišine i govora,najduža priča,zcr prosek,zcr mean,zcr max,zcr med,zcr stdev,pitch prosek,pitch mean,pitch max,pitch med,pitch stdev,energija prosek,energija min,energija max,energija stdev,eng bez tis,eng bez tis avg,eng bez tis min,eng bez tis max,eng bez tis med,eng bez tis stdev,energija ukupna,mfccovi x,mfccovi y,mfccovi
0,03a01Fa.wav,3,a01,F,Sreća,M,31,0.057822,0.027695,0.014496,0.016678,0.039627,0.000000,0.041737,0.035844,0.040875,0.034708,0.031669,0.035054,0.034017,0.042673,0.017427,0.044139,0.032203,0.011907,0.025037,0.014647,0.002583,0.032203,0.007222,0.023040,0.009818,20,82,-573.01196;-582.12134;-586.8176;-584.4865;-338...
1,03a01Nc.wav,3,a01,N,Neutralno,M,31,0.051544,0.023508,0.012098,0.013819,0.027523,0.000000,0.034284,0.022568,0.032566,0.025073,0.027890,0.022468,0.027078,0.026987,0.042682,0.042051,0.043451,0.036026,0.042954,0.033954,0.000579,0.043451,0.009349,0.040164,0.019309,20,70,-544.9087;-524.06104;-542.695;-466.05853;-281....
2,03a01Wa.wav,3,a01,W,Bes,M,31,0.051150,0.027396,0.014676,0.016917,0.050864,0.100504,0.038425,0.063722,0.038340,0.039823,0.047725,0.032331,0.041398,0.028820,0.041732,0.042269,0.045070,0.024851,0.044305,0.035076,0.000232,0.045070,0.017133,0.041947,0.023258,20,81,-545.2024;-542.0179;-541.4934;-541.7991;-532.3...
3,03a02Fc.wav,3,a02,F,Sreća,M,31,0.048770,0.029270,0.015113,0.033119,0.042765,0.000000,0.046209,0.043809,0.040676,0.049342,0.045570,0.049004,0.046737,0.051125,0.039837,0.048514,0.055141,0.027702,0.043035,0.033512,0.002959,0.055141,0.015307,0.040674,0.023722,20,87,-462.67767;-450.47595;-474.7913;-311.72556;-17...
4,03a02Nc.wav,3,a02,N,Neutralno,M,31,0.047226,0.021006,0.019172,0.027877,0.027608,0.000000,0.036437,0.023896,0.032225,0.023681,0.028542,0.019003,0.025484,0.018476,0.057703,0.042335,0.052075,0.034992,0.058653,0.052259,0.007179,0.052075,0.018944,0.057258,0.026543,20,63,-544.0373;-531.3774;-527.38007;-405.45212;-268...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,16b10Lb.wav,16,b10,L,Dosada,F,31,0.049579,0.050227,0.034127,0.047772,0.037308,0.075378,0.043890,0.033189,0.039550,0.036696,0.042519,0.040970,0.032669,0.040968,0.034995,0.044069,0.039598,0.016916,0.040155,0.035540,0.002804,0.039598,0.017168,0.040335,0.043180,20,149,-463.54443;-465.73322;-494.0283;-500.79193;-46...
531,16b10Tb.wav,16,b10,T,Tuga,F,31,0.037642,0.051072,0.018778,0.060281,0.038834,0.000000,0.051343,0.029206,0.044661,0.041108,0.057747,0.038671,0.040220,0.017127,0.078883,0.042005,0.044063,0.058161,0.069282,0.071198,0.047161,0.044063,0.032171,0.068801,0.087927,20,151,-445.80164;-435.16284;-438.70038;-427.3457;-19...
532,16b10Td.wav,16,b10,T,Tuga,F,31,0.048029,0.057398,0.016357,0.058732,0.030288,0.000000,0.047203,0.018586,0.039758,0.033811,0.047177,0.026413,0.033626,0.012115,0.035342,0.042027,0.032165,0.033624,0.035049,0.030735,0.038350,0.032165,0.015337,0.033847,0.042653,20,170,-444.67496;-447.99628;-452.18787;-215.87308;-1...
533,16b10Wa.wav,16,b10,W,Bes,F,31,0.031072,0.035221,0.013295,0.042292,0.062608,0.175882,0.038756,0.086290,0.035835,0.058613,0.046367,0.063184,0.060262,0.071997,0.008845,0.042101,0.014207,0.003565,0.013078,0.007176,0.000096,0.014207,0.002408,0.011827,0.006113,20,104,-613.46625;-595.7472;-595.0384;-627.9332;-568....
